In [1]:
import keyring

keyring.set_password('tiingo', 'myeongsu', '409b19bf9cce62b4463b8baf2d9300fb6c835b86')

## 전 세계 주식 데이터 수집
[Tiingo](https://api.tiingo.com/) 사이트를 접속해 데이터를 수집한다.

#### 데이터 다운로드
- 데이터를 받기 위해 API 접속 환경을 세팅한다.

In [2]:
from tiingo import TiingoClient
import pandas as pd
import keyring

api_key = keyring.get_password('tiingo', 'myeongsu')
config = {}
config['session'] = True
config['api_key'] = api_key
client = TiingoClient(config)

- Tiingo에서 제공하는 종목은 어떠한 것이 있는지 티커 정보들을 확인한다.

In [3]:
tickers = client.list_stock_tickers()
tickers_df = pd.DataFrame.from_records(tickers)

tickers_df.head()

,ticker,exchange,assetType,priceCurrency,startDate,endDate
0,-P-H,NYSE,Stock,USD,,
1,-P-S,NYSE,Stock,USD,2018-08-22,2023-05-05
2,000001,SHE,Stock,CNY,2007-01-04,2025-03-21
3,000002,SHE,Stock,CNY,2007-01-04,2025-03-21
4,000003,SHE,Stock,CNY,,


- 통화별 종목이 몇 개 있는지 확인해본다.

In [4]:
tickers_df.groupby(['exchange', 'priceCurrency'])['ticker'].count()

exchange   priceCurrency
           USD               2452
AMEX       USD                112
ASX        AUD                163
           USD               2115
BATS       USD                 54
CSE        USD                 32
EXPM       USD               2937
LSE        USD                 12
NASDAQ     USD              12952
NMFQS      USD                 36
NYSE       USD               7843
NYSE ARCA  USD                114
NYSE MKT   USD                464
NYSE NAT   USD                  3
OTCBB      USD                646
OTCCE      USD               1090
OTCGREY    USD               4012
OTCMKTS    USD               1190
OTCQB      USD               1231
OTCQX      USD                763
PINK       USD              15525
SHE        CNY               3386
           HKD                 12
SHEB       HKD                 42
SHG        CNY               2954
           USD                  6
SHGB       USD                 44
Name: ticker, dtype: int64

- 특정 종목의 상세 정보를 확인해 본다. (APPLE)

In [5]:
ticker_metadata = client.get_ticker_metadata("AAPL")
print(ticker_metadata)

{'ticker': 'AAPL', 'name': 'Apple Inc', 'description': "Apple Inc. (Apple) designs, manufactures and markets mobile communication and media devices, personal computers, and portable digital music players, and a variety of related software, services, peripherals, networking solutions, and third-party digital content and applications. The Company's products and services include iPhone, iPad, Mac, iPod, Apple TV, a portfolio of consumer and professional software applications, the iOS and OS X operating systems, iCloud, and a variety of accessory, service and support offerings. The Company also delivers digital content and applications through the iTunes Store, App StoreSM, iBookstoreSM, and Mac App Store. The Company distributes its products worldwide through its retail stores, online stores, and direct sales force, as well as through third-party cellular network carriers, wholesalers, retailers, and value-added resellers. In February 2012, the Company acquired app-search engine Chomp.", 

- 주가를 받아본다.

In [7]:
historical_prices = client.get_dataframe("AAPL",
                                        startDate='2017-08-01',
                                        frequency='daily')
historical_prices.head()

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
2017-08-01 00:00:00+00:00,150.05,150.22,148.4100,149.10,24725526,34.991452,35.031096,34.609006,34.769913,98902104,0.0,1.0
2017-08-02 00:00:00+00:00,157.14,159.75,156.1600,159.28,69222793,36.644830,37.253479,36.416296,37.143875,276891172,0.0,1.0
2017-08-03 00:00:00+00:00,155.57,157.21,155.0200,157.05,26000738,36.278709,36.661154,36.150449,36.623842,104002952,0.0,1.0
2017-08-04 00:00:00+00:00,156.39,157.40,155.6900,156.07,20349532,36.469931,36.705462,36.306692,36.395308,81398128,0.0,1.0
2017-08-07 00:00:00+00:00,158.81,158.92,156.6701,157.06,21870321,37.034272,37.059924,36.535250,36.626174,87481284,0.0,1.0


- 일별 가치지표를 받아 본다. (무료 계정의 경우 다우존스 30 지수에 포함되는 종목 정보만 제공한다.)

In [8]:
fundamentals_daily = client.get_fundamentals_daily("AAPL")
fundamentals_daily_df = pd.DataFrame.from_records(fundamentals_daily)

fundamentals_daily_df.head()

,date,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y
0,2022-03-24T00:00:00.000Z,2.855862e+12,2.914747e+12,28.400990,39.702240,1.177602
1,2022-03-25T00:00:00.000Z,2.866526e+12,2.925411e+12,28.507043,39.850493,1.181999
2,2022-03-26T00:00:00.000Z,NaN,NaN,28.121113,42.530686,3.050059
3,2022-03-28T00:00:00.000Z,2.865694e+12,2.934164e+12,28.112953,42.518344,3.049174
4,2022-03-29T00:00:00.000Z,2.920527e+12,2.988997e+12,28.650877,43.331906,3.107518


- 재무제표를 받아본다.

In [9]:
fundamentals_stmnts = client.get_fundamentals_statements(   # 1
    'AAPL', startDate='2019-01-01', asReported=True, fmt='csv')

df_fs = pd.DataFrame([x.split(',') for x in fundamentals_stmnts.split('\n')])   # 2
df_fs.columns = df_fs.iloc[0]   # 3
df_fs = df_fs[1:]   # 3
df_fs.set_index('date', drop=True, inplace=True)   # 4
df_fs = df_fs[df_fs.index != '']   # 5

df_fs.head()

,year,quarter,statementType,dataCode,value
date,,,,,
2025-01-31,2025,1,cashFlow,depamor,3080000000.0
2025-01-31,2025,1,balanceSheet,ppeq,46069000000.0
2025-01-31,2025,1,balanceSheet,intangibles,0.0
2025-01-31,2025,1,incomeStatement,opinc,42832000000.0
2025-01-31,2025,1,incomeStatement,shareswa,15081724000.0


1. get_fundamentals_statements() 메서드 내에 티커를 입력하면 재무제표의 세부 항목을 받을 수 있다. 또한, fmt는 포맷 형태를 의미하며, JSON으로 받을 경우 형태가 지나치게 복잡하므로 CSV로 받는 것이 좋다.
2. 텍스트 형태로 데이터가 들어오므로, 클렌징을 통해 데이터프레임 형태로 변경한다.
3. 첫 번째 행을 열 이름으로 지정한 후, 해당 행은 삭제한다.
4. 'date' 열을 인덱스로 지정한다.
5. 'date'가 비어 있는 부분이 있으므로 이를 제거한다.